# Account - Owner Services PROD

## Database Summary

In [1]:
Import-Module AWS.Tools.Common, AWS.Tools.DynamoDBv2, AWS.Tools.RDS
Import-Module '/Users/conrad.gauntlett/WorkArea/Repos/DBA_MISC/PowerShell/Modules/DevOpsToolkit'

$Account            = 'DevOps-Owner-Services-PROD'
$RDSSummaryColumns  = 'AvailabilityZone', 'DBSnapshotIdentifier', 'SnapshotCreateTime', 'SnapshotType', 'Status'
$Profiles           = Get-AWSCredentials -ListProfileDetail | Where-Object { $_.ProfileName -match $Account }
$Regions            = "eu-central-1" , "eu-west-2" , "eu-west-1", "us-east-1"
$Summary            = Get-DoAWSDBInformation $Profiles $Regions
$Summary | Format-Table -AutoSize


Account             AccountID    Region    RDSInstances DynamoDBTables
-------             ---------    ------    ------------ --------------
Owner-Services-PROD 496175963148 eu-west-2            3              1



### DynamoDB Tables

In [2]:
$ddbTables = Get-DoDDBTableInformation $Profiles $Regions 
$ddbTables | Format-Table -AutoSize


Region    TableName       TableBackup CreationDateTime      ItemCount TableSizeMB
------    ---------       ----------- ----------------      --------- -----------
eu-west-2 terraform-locks       False 12/12/2022 1:05:57 PM        70 0 MB



### RDS Instances

In [3]:
$RDS        = Get-DoRDSDBSummary $Profiles $Regions 
$Snapshots  = Get-DoRDSSnapshotSummary $Profiles $Regions
$RDS | Format-Table -AutoSize


Region    DBInstanceIdentifier Engine   Version InstanceCreateTime   Snapshots Retention
------    -------------------- ------   ------- ------------------   --------- ---------
eu-west-2 authenticationdbprod postgres 16.1    3/12/2024 8:57:35 AM        10         7
eu-west-2 bookingdbprod        postgres 16.1    3/12/2024 9:02:23 AM        10         7
eu-west-2 notificationdbprod   postgres 16.3    3/12/2024 8:57:15 AM        10         7



#### Snapshots

##### authenticationdbprod

In [4]:
$DB = 'authenticationdbprod'
$Snapshots | Where-Object {$_.DBInstanceIdentifier -eq $DB} | Select-Object $RDSSummaryColumns |
Sort-Object SnapshotCreateTime -Descending  |Format-Table -AutoSize


AvailabilityZone DBSnapshotIdentifier                      SnapshotCreateTime   SnapshotType Status
---------------- --------------------                      ------------------   ------------ ------
eu-west-2c       rds:authenticationdbprod-2024-07-15-02-56 7/15/2024 3:56:45 AM automated    avail…
eu-west-2c       rds:authenticationdbprod-2024-07-14-02-56 7/14/2024 3:56:58 AM automated    avail…
eu-west-2c       rds:authenticationdbprod-2024-07-13-02-56 7/13/2024 3:56:42 AM automated    avail…
eu-west-2c       rds:authenticationdbprod-2024-07-12-02-57 7/12/2024 3:57:18 AM automated    avail…
eu-west-2c       rds:authenticationdbprod-2024-07-11-02-56 7/11/2024 3:56:46 AM automated    avail…
eu-west-2c       rds:authenticationdbprod-2024-07-10-02-56 7/10/2024 3:56:55 AM automated    avail…
eu-west-2c       rds:authenticationdbprod-2024-07-09-02-56 7/9/2024 3:56:51 AM  automated    avail…
eu-west-2c       rds:authenticationdbprod-2024-07-08-02-56 7/8/2024 3:56:40 AM  automated    avail…

##### bookingdbprod

In [5]:
$DB = 'bookingdbprod'
$Snapshots | Where-Object {$_.DBInstanceIdentifier -eq $DB} | Select-Object $RDSSummaryColumns |
Sort-Object SnapshotCreateTime -Descending  |Format-Table -AutoSize


AvailabilityZone DBSnapshotIdentifier               SnapshotCreateTime   SnapshotType Status
---------------- --------------------               ------------------   ------------ ------
eu-west-2c       rds:bookingdbprod-2024-07-15-04-56 7/15/2024 5:56:18 AM automated    available
eu-west-2c       rds:bookingdbprod-2024-07-14-04-56 7/14/2024 5:56:49 AM automated    available
eu-west-2c       rds:bookingdbprod-2024-07-13-04-56 7/13/2024 5:56:42 AM automated    available
eu-west-2c       rds:bookingdbprod-2024-07-12-04-56 7/12/2024 5:56:45 AM automated    available
eu-west-2c       rds:bookingdbprod-2024-07-11-04-56 7/11/2024 5:56:58 AM automated    available
eu-west-2c       rds:bookingdbprod-2024-07-10-04-56 7/10/2024 5:56:56 AM automated    available
eu-west-2c       rds:bookingdbprod-2024-07-09-04-56 7/9/2024 5:56:49 AM  automated    available
eu-west-2c       rds:bookingdbprod-2024-07-08-04-56 7/8/2024 5:56:57 AM  automated    available
eu-west-2c       rds:bookingdbprod-2024-07-07

##### notificationdbprod

In [6]:
$DB = 'notificationdbprod'
$Snapshots | Where-Object {$_.DBInstanceIdentifier -eq $DB} | Select-Object $RDSSummaryColumns |
Sort-Object SnapshotCreateTime -Descending  |Format-Table -AutoSize


AvailabilityZone DBSnapshotIdentifier                    SnapshotCreateTime   SnapshotType Status
---------------- --------------------                    ------------------   ------------ ------
eu-west-2b       rds:notificationdbprod-2024-07-15-00-20 7/15/2024 1:20:38 AM automated    availab…
eu-west-2b       rds:notificationdbprod-2024-07-14-00-20 7/14/2024 1:20:45 AM automated    availab…
eu-west-2b       rds:notificationdbprod-2024-07-13-00-20 7/13/2024 1:20:33 AM automated    availab…
eu-west-2b       rds:notificationdbprod-2024-07-12-00-20 7/12/2024 1:20:25 AM automated    availab…
eu-west-2b       rds:notificationdbprod-2024-07-11-00-20 7/11/2024 1:20:29 AM automated    availab…
eu-west-2b       rds:notificationdbprod-2024-07-10-00-20 7/10/2024 1:20:46 AM automated    availab…
eu-west-2b       rds:notificationdbprod-2024-07-09-00-20 7/9/2024 1:20:43 AM  automated    availab…
eu-west-2b       rds:notificationdbprod-2024-07-08-00-21 7/8/2024 1:21:16 AM  automated    availab…
eu-